In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os 
import glob
tqdm.pandas()
import matplotlib.pyplot as plt
import gc

In [ ]:
filepath = "../input/ubiquant-market-prediction/train.csv"
nrows = 50000
df_train = pd.read_csv(filepath, nrows=nrows)

In [ ]:
df_train.head()

In [ ]:
df_train["time_id"].value_counts()

In [ ]:
df_train["investment_id"].value_counts()

In [ ]:
X = df_train.groupby("investment_id")
X.size().sort_values()

In [ ]:
num_features = list(df_train.filter(like="f_").columns)
features = num_features

In [ ]:
# find the correlation coefficient between features and target
df=df_train
cor_t=df[features].apply(lambda x: x.corr(df['target']))
cr=pd.DataFrame(columns = ['feature','correlation'],)
cr['feature'] = pd.Series(features)
cr['correlation'] = pd.Series(list(abs(cor_t)))
temp=cr.sort_values(by="correlation", ascending=False ).head(100)
print(temp.head(20))
features_first100=list(temp['feature'])
df_train_filtered=df_train[features_first100]
del df
df_train_filtered.shape

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
##X = df_train_filtered[features_first100]
X = df_train[features]
y = df_train['target']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)
del X,y
from xgboost.sklearn import XGBRegressor
xgb_model = XGBRegressor(objective='reg:squarederror',n_estimators=100, max_depth=8, learning_rate=0.1, random_state=0)
xgb_model.fit(X_train, y_train)
y_predict = xgb_model.predict(X_test)

score = xgb_model.score(X_train, y_train)  
print("Training score: ", score)
mse = mean_squared_error(y_test,y_predict)
print("MSE: ", mse)
#scores = cross_val_score(xgb_model, X_train, y_train)
#print("Mean cross-validation score: %.2f" % scores.mean())

#kfold = KFold(n_splits=5, shuffle=True)
#kf_cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=kfold )
#print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

#sorted_idx = xgb_model.feature_importances_.argsort()
#plt.barh(X_train.columns[sorted_idx], xgb_model.feature_importances_[sorted_idx])
#plt.xlabel("Xgboost Feature Importance")
features_importance_df= pd.DataFrame({'feature': features, 'importance': xgb_model.feature_importances_}).sort_values(by="importance", ascending=False)
plt.figure(figsize=(14, 8))
plt.subplot(1,2,1)
sns.barplot(x="importance", y="feature", data=features_importance_df.head(25))
plt.title(f'50 Head XGBRegressor Features ')
plt.subplot(1,2,2)
sns.barplot(x="importance", y="feature", data=features_importance_df.tail(25))
plt.title(f'50 Tail XGBRegressor Features ')
plt.tight_layout()
plt.show()                              

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
    final_pred =xgb_model.predict(test_df[features]) 
    sample_prediction_df['target'] = final_pred
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)